### **1. Install the required dependencies**

In [ ]:
!pip install ultralytics
!git clone https://github.com/ultralytics/yolov5  # clone
!cd yolov5
!pip install -r requirements.txt  # install
!pip install clearml


In [ ]:
import torch
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torchvision.transforms as transforms
from PIL import Image
import yaml
import os
import shutil
import random

In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
API_ACCESS_KEY = user_secrets.get_secret("CLEARML_API_ACCESS_KEY")
API_SECRET_KEY = user_secrets.get_secret("CLEARML_API_SECRET_KEY")

In [ ]:
%env CLEARML_WEB_HOST=https://app.clear.ml
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml
%env CLEARML_API_ACCESS_KEY=API_ACCESS_KEY
%env CLEARML_API_SECRET_KEY=API_SECRET_KEY

In [9]:
from clearml import Task
task = Task.init(project_name="Bank Note Detection and Classification", task_name="Detection 250 epochs")
  



/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


ClearML Task: created new task id=0db7f991719e44b586f76e259125a008
2023-05-21 22:58:03,350 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/ad200e2164044bcfb399eb7335e7081b/experiments/0db7f991719e44b586f76e259125a008/output/log


In [11]:
dataset_path = "/kaggle/input/euro-banknotes/dataset"
images_path = os.path.join(dataset_path, "data/images")
labels_path = os.path.join(dataset_path, "data/labels")
data_yaml_path = os.path.join(dataset_path, "data.yaml")

In [12]:
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

In [13]:
output_path = "/kaggle/working/dataset"
train_path = os.path.join(output_path, "train")
val_path = os.path.join(output_path, "val")
test_path = os.path.join(output_path, "test")


In [14]:
os.makedirs(train_path, exist_ok=True)
os.makedirs(val_path, exist_ok=True)
os.makedirs(test_path, exist_ok=True)

# Load data.yaml
with open(data_yaml_path, 'r') as file:
    data_yaml = yaml.load(file, Loader=yaml.FullLoader)

# Get all image and label filenames
image_files = os.listdir(images_path)
label_files = os.listdir(labels_path)

# Shuffle the filenames
random.shuffle(image_files)

# Calculate split indices
train_split = int(len(image_files) * train_ratio)
val_split = int(len(image_files) * (train_ratio + val_ratio))

# Split image and label filenames
train_images = image_files[:train_split]
val_images = image_files[train_split:val_split]
test_images = image_files[val_split:]

# Move images to corresponding directories
for image in train_images:
    shutil.copy(os.path.join(images_path, image), train_path)
for image in val_images:
    shutil.copy(os.path.join(images_path, image), val_path)
for image in test_images:
    shutil.copy(os.path.join(images_path, image), test_path)

# Move labels to corresponding directories
for image in train_images:
    shutil.copy(os.path.join(labels_path, image.replace(".jpg", ".txt")), train_path)
for image in val_images:
    shutil.copy(os.path.join(labels_path, image.replace(".jpg", ".txt")), val_path)
for image in test_images:
    shutil.copy(os.path.join(labels_path, image.replace(".jpg", ".txt")), test_path)

In [15]:
data_yaml = """
train: /kaggle/working/dataset/train
val: /kaggle/working/dataset/val
test: /kaggle/working/dataset/test

nc: 1  # Number of classes
names: ['0']
"""

# Set the file path to save the data.yaml
data_yaml_path = '/kaggle/working/dataset/data.yaml'

# Save the data.yaml file
with open(data_yaml_path, 'w') as file:
    file.write(data_yaml)

In [16]:
# Load data.yaml
with open(data_yaml_path, 'r') as file:
    data_yaml = yaml.load(file, Loader=yaml.FullLoader)

In [ ]:
!python yolov5/train.py --img 640 --batch 32 --epochs 250 --data /kaggle/working/dataset/data.yaml --weights /kaggle/working/yolov5x.pt


In [18]:
!python yolov5/val.py --img 640 --data /kaggle/working/dataset/data.yaml --weights yolov5/runs/train/exp/weights/best.pt --half

val: data=/kaggle/working/dataset/data.yaml, weights=['yolov5/runs/train/exp/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=yolov5/runs/val, name=exp, exist_ok=False, half=True, dnn=False
YOLOv5 🚀 v7.0-169-geef637c Python-3.10.10 torch-2.0.0 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 322 layers, 86173414 parameters, 0 gradients, 203.8 GFLOPs
val: Scanning /kaggle/working/dataset/val.cache... 40 images, 3 backgrounds, 0 c
                 Class     Images  Instances          P          R      mAP50   
                   all         40        185      0.961      0.962      0.966      0.938
Speed: 0.2ms pre-process, 31.5ms inference, 2.2ms NMS per image at shape (32, 3, 640, 640)
Results saved to yolov5/runs/val/exp


In [19]:
!python yolov5/val.py --weights yolov5/runs/train/exp/weights/best.pt --data /kaggle/working/dataset/data.yaml --img 832 --augment --half

val: data=/kaggle/working/dataset/data.yaml, weights=['yolov5/runs/train/exp/weights/best.pt'], batch_size=32, imgsz=832, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=True, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=yolov5/runs/val, name=exp, exist_ok=False, half=True, dnn=False
YOLOv5 🚀 v7.0-169-geef637c Python-3.10.10 torch-2.0.0 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 322 layers, 86173414 parameters, 0 gradients, 203.8 GFLOPs
val: Scanning /kaggle/working/dataset/val.cache... 40 images, 3 backgrounds, 0 c
                 Class     Images  Instances          P          R      mAP50   
                   all         40        185      0.955      0.957      0.953      0.916
Speed: 0.4ms pre-process, 94.6ms inference, 1.4ms NMS per image at shape (32, 3, 832, 832)
Results saved to yolov5/runs/val/exp2


In [23]:
!python yolov5/detect.py --weights yolov5/runs/train/exp/weights/best.pt --source /kaggle/input/euro-banknotes/dataset/data/images --img 640 --half --save-txt --save-crop

segment/predict: weights=['yolov5/runs/train-seg/exp/weights/best.pt'], source=/kaggle/input/euro-banknotes/dataset/data/images, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=True, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/predict-seg, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=True, dnn=False, vid_stride=1, retina_masks=False
YOLOv5 🚀 v7.0-169-geef637c Python-3.10.10 torch-2.0.0 CUDA:0 (Tesla T4, 15110MiB)

Traceback (most recent call last):
  File "/kaggle/working/yolov5/segment/predict.py", line 284, in <module>
    main(opt)
  File "/kaggle/working/yolov5/segment/predict.py", line 279, in main
    run(**vars(opt))
  File "/opt/conda/lib/python3.10/site-packages/torch/utils/_contextlib.py", line 115, in decorate_context
    return func(*args, **kwargs)
  F